Imports

In [102]:
import cv2
import pandas as pd
import os
import numpy as np

# načítaj csv s Ground truth
data_folder = 'duhovky/'
df = pd.read_csv(os.path.join(data_folder, 'iris_annotation.csv'))

# prechádzaj po riadkov z csv
for index, row in df.iterrows():
    # načítaj obrázok
    img = cv2.imread(os.path.join(data_folder, row['image']))

    scale_percent = 125 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    # resize image
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    
    # Convert to grayscale.
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = cv2.equalizeHist(img)

    # img = cv2.GaussianBlur(img, (5, 5), 0)

    #img = cv2.fastNlMeansDenoising(img, None, 31, 7, 21) 

    clahe = cv2.createCLAHE(clipLimit=2)
    img = clahe.apply(img) + 30

    # img = cv2.GaussianBlur(img, (5, 5), 0)

    #img = cv2.fastNlMeansDenoising(img, None, 31, 7, 21) 

    # ret, img = cv2.threshold(img, 107, 215, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    img = cv2.fastNlMeansDenoising(img, None, 31, 7, 21) 

    img = cv2.Canny(img, 0, 255)
    
    # Apply Hough transform on the blurred image.
    detected_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 1, 20,
                                    param1=150, param2=55,
                                    minRadius=55, maxRadius=200)
    
    # Draw circles that are detected.
    if detected_circles is not None:
    
        # Convert the circle parameters a, b and r to integers.
        detected_circles = np.uint16(np.around(detected_circles))
    
        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]
    
            # Draw the circumference of the circle.
            cv2.circle(img, (a, b), r, (250, 255, 255), 4)
    
            # Draw a small circle (of radius 1) to show the center.
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3)

    cv2.imshow('annotation-test', img)
    # zobraz obrázok, počak 500 ms
    c = cv2.waitKey(500)
    # pri stlačení Esc break
    if c == 27:
        break

cv2.destroyAllWindows()